In [1]:
import pandas as pd
import math

/home/sme777/.pyenv/versions/3.7.0/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [ ]:
class PickListMaker:
    
    def __init__(self, filenames, solution_moles, solution_volume, \
                 buffer_moles, origami=True, scaffold="M13mp18", ratio="10:1", \
                 solution_well="I10", skip=[None, None], skip_buffer=False, skip_scaffold=True):
        self.scaffold_molarity = {"M13mp18": 0.21e-12/1e-6}
        if len(filenames) != 1 and skip == [None, None]:
            self.skip = [[None, None] for _ in range(len(filenames))]
        else:
            self.skip = [skip]
        self.strands_df = self.generates_dataframes(filenames)
        self.solution_moles = solution_moles
        self.solution_volume = solution_volume
        self.origami = origami
        self.solution_well = solution_well
        self.buffer_moles = buffer_moles
        self.scaffold = scaffold
        self.skip_buffer = skip_buffer
        self.skip_scaffold = skip_scaffold
        self.ratio = float(ratio.split(":")[0]) if origami else 1
        self.picklist_df = self.generate_picklist(self.strands_df)

    def generates_dataframes(self, filenames):
        strands_dfs = []
        for idx, filename in enumerate(filenames):
            file_extn = filename.split(".")[-1]
            if file_extn == "xlsx":
                full_df = pd.read_excel(filename,engine='openpyxl')
                if self.skip[idx] != [None, None]:
                    filtered_df = pd.concat([full_df.iloc[:self.skip[idx][0], :], new_df.iloc[self.skip[idx][1]:, :]])
                else:
                    filtered_df = full_df
                strands_dfs.append(filtered_df)
            elif file_extn == "csv":
                full_df = pd.read_csv(filename)
                if self.skip[idx] != [None, None]:
                    filtered_df = pd.concat([full_df.iloc[:self.skip[idx][0], :], new_df.iloc[self.skip[idx][1]:, :]])
                else:
                    filtered_df = full_df
                strands_dfs.append(filtered_df)
        return strands_dfs

    def split_by_plate_name(self, staples):
        split_staples = []
        for staple_group in staples:
            for name, df in staple_group.groupby('Plate Name'):
                split_staples.append(df)
        return split_staples
    
    def generate_picklist(self, all_staples):
        source_plate_name = []
        source_plate_type = []
        source_well_positions = []
        sample_comments = []
        destination_plate_name = []
        destination_wells = []
        transfer_volumes = []
        all_staples = self.split_by_plate_name(all_staples)
        if self.origami:
            scaffold_transfer_volume = (self.solution_moles * self.solution_volume) / self.scaffold_molarity[self.scaffold]
        else:
            scaffold_transfer_volume = 0
        staple_concentrations = [staples_group["Measured Concentration µM "].mean() for staples_group in all_staples]
        staples_num = sum([len(staples_group) for staples_group in all_staples])
        staple_concentrations_avg = math.floor(sum(staple_concentrations) / len(staple_concentrations))*1e-6
        staple_transfer_volume = (self.ratio * 
                                  self.solution_moles * self.solution_volume) / staple_concentrations_avg
        staple_transfer_volume = int(round(staple_transfer_volume*1e+9, 0)) # in nanoliters
        dna_volume = scaffold_transfer_volume + staples_num * staple_transfer_volume*1e-9
        buffer_volume = self.solution_volume - dna_volume
        magnesium_volume = self.solution_volume * self.buffer_moles
        print("Average Staple Concetration is: {:.1f}µM with transfer volume {:.1f}nl.".format(round(staple_concentrations_avg*1e+6, 0), staple_transfer_volume))
        print("Total DNA volume is {:.2f}µl and buffer volume is {:.2f}µl.".format(dna_volume*1e+6, buffer_volume*1e+6))
        print("Magnesium volume is {:.2f}µl. This will be placed in the last available well.".format(magnesium_volume*1e+6))
        
        for idx, staple_pkg in enumerate(all_staples):
            for item, row in staple_pkg.iterrows():
                source_plate_name.append(f"Source[{idx+1}]")
                source_plate_type.append("384PP_AQ_BP")
                last_well_position = row["Well Position"]
                source_well_positions.append(last_well_position)
                sample_comments.append(row["Sequence"])
                destination_plate_name.append("Destination[1]")
                destination_wells.append(self.solution_well)
                transfer_volumes.append(staple_transfer_volume)
                
        if not self.skip_buffer:
            source_plate_name.append(f"Source[{idx+1}]")
            source_plate_type.append("384PP_AQ_BP")
            magnesium_well_position = last_well_position

            if int(magnesium_well_position[1:]) > 24:
                letter_ord = (ord(s.upper())+1 - 65) % 26 + 65
                if letter_ord > 80:
                    print("Ran out of well to place buffer, please pipet manually.")
                else:
                    magnesium_well_position[0] = chr(letter_ord)
            else:
                magnesium_well_position = magnesium_well_position[0] + str(int(magnesium_well_position[1:]) + 1).zfill(2)



            if magnesium_well_position != last_well_position:
                source_well_positions.append(magnesium_well_position)
                sample_comments.append("Mg")
                destination_plate_name.append("Destination[1]")
                destination_wells.append("I10")
                transfer_volumes.append(magnesium_volume*1e+9)

        if self.origami and not self.skip_scaffold:
            source_plate_name.append(f"Source[{idx+1}]")
            source_plate_type.append("384PP_AQ_BP")
            scaffold_well_position = magnesium_well_position
            if int(scaffold_well_position[1:]) > 24:
                letter_ord = (ord(scaffold_well_position[0].upper())+1 - 65) % 26 + 65
                if letter_ord > 80:
                    print("Ran out of well to place buffer, please pipet manually.")
                else:
                    scaffold_well_position[0] = chr(letter_ord)
            else:
                scaffold_well_position = scaffold_well_position[0] + str(int(scaffold_well_position[1:]) + 1).zfill(2)


            if scaffold_well_position != magnesium_well_position:
                source_well_positions.append(scaffold_well_position)
                sample_comments.append(self.scaffold)
                destination_plate_name.append("Destination[1]")
                destination_wells.append("I10")
                transfer_volumes.append(scaffold_transfer_volume*1e+9)
        echo_df = pd.DataFrame({"Source Plate Name": source_plate_name, "Source Plate Type": source_plate_type, \
                           "Source Well": source_well_positions, "Sample Comments": sample_comments, \
                           "Destination Plate Name": destination_plate_name, "Destination Well": destination_wells, \
                           "Transfer Volume": transfer_volumes})
        echo_df["Transfer Volume"] = echo_df["Transfer Volume"].astype('int')
        echo_df = echo_df.set_index("Source Plate Name")
        return echo_df
    
    def save_to_csv(self, path):
        self.picklist_df.to_csv(path)
        
    def save_to_excel(self, path):
        self.picklist_df.to_excel(path)

In [ ]:
#####################################
#####################################
#####################################
### Example Experiment Parameters ###
#####################################
#####################################
#####################################

mix_moles = 10e-9
mix_volume = 50e-6
buffer_moles = 10e-3
picklist = PickListMaker(["./nanoframe_staples.xlsx"], mix_moles, mix_volume, buffer_moles)

In [ ]:
### Visualize the Generated Picklist ###
picklist.picklist_df

In [ ]:
### Save the Generated Picklist to CSV ###
picklist.save_to_csv("./nanoframe_picklist.csv")

In [ ]:
### Save the Generated Picklist to Excel ###
picklist.save_to_excel("./nanoframe_picklist.xlsx")